In [1]:
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt;
import logging;
import os;
import time;
from datetime import date, datetime;
from flood_pred_data_processing_V1 import DataProcessor;
from flood_pred_models_V1 import FloodModel;
import seaborn as sns 


Processor = DataProcessor() 
surrey_df = Processor.process_surrey()
# print(surrey_df)
wu_df = Processor.concat_multiple_wu_stations()
# print (wu_df)

Model = FloodModel()  
model_df = Processor.build_model_matrix() 
# model_df.describe().transpose()[['mean', 'std']]

features_df = model_df.drop(columns=['water_level'])
# Normalize the data
normalizer = Model.model_data_normalizer(features_df)
normalized_features = normalizer(features_df.to_numpy())

# Reattach the target column
normalized_data = np.concatenate([normalized_features, model_df[['water_level']].to_numpy()], axis=1)
# Preparing train and test data for LSTM
n_past = 48  # e.g., use 24hrs past time steps to predict
n_future = 16  # e.g., predict 4 hours ahead
# n_features = 50
X_train, y_train = Model.create_lstm_dataset(normalized_data, n_future, n_past)
X_test, y_test = Model.create_lstm_dataset(normalized_data, n_future, n_past)

n_features = normalized_data.shape[1]  # Assuming all columns except the target are features
input_shape = (n_past, n_features)

# Build the LSTM model
model = Model.build_lstm_model(input_shape)
history = Model.train_model(model, X_train, y_train, epochs=50, batch_size=32)
predictions = Model.predict_future(model, X_test)


X_train, y_train = Model.create_lstm_dataset(normalized_data, n_future, n_past)
X_test, y_test = Model.create_lstm_dataset(normalized_data, n_future, n_past)

# Building and training the LSTM model
lstm_model_wo_water = Model.build_lstm_model((X_train.shape[1], X_train.shape[2]))
history = Model.train_model(lstm_model_wo_water, X_train, y_train, epochs=50)

# Predicting future values
predictions_wo_water = Model.predict_future(lstm_model_wo_water, X_test)

X_train, y_train = Model.create_lstm_dataset_include_water(normalized_data, n_future, n_past)
X_test, y_test = Model.create_lstm_dataset_include_water(normalized_data, n_future, n_past)




The dataframe is being concatenated


2024-01-15 18:11:19.325053: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/50
188/188 [==============================] - 3s 12ms/step - loss: 0.7678 - val_loss: 0.2319
Epoch 2/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0219 - val_loss: 0.2059
Epoch 3/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0162 - val_loss: 0.2079
Epoch 4/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0133 - val_loss: 0.2116
Epoch 5/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0137 - val_loss: 0.2020
Epoch 6/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0124 - val_loss: 0.2014
Epoch 7/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0114 - val_loss: 0.1993
Epoch 8/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0113 - val_loss: 0.1895
Epoch 9/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0118 - val_loss: 0.1814
Epoch 10/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0101 - val_lo

Epoch 31/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0079 - val_loss: 0.1253
Epoch 32/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0069 - val_loss: 0.1200
Epoch 33/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0088 - val_loss: 0.1231
Epoch 34/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0073 - val_loss: 0.1205
Epoch 35/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0083 - val_loss: 0.1229
Epoch 36/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0066 - val_loss: 0.1144
Epoch 37/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0063 - val_loss: 0.1142
Epoch 38/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0058 - val_loss: 0.1158
Epoch 39/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0059 - val_loss: 0.1181
Epoch 40/50
188/188 [==============================] - 2s 10ms/step - loss: 0.0059

In [2]:
# Evaluate the model
mae, mse, rmse, r2 = Model.evaluate_model(model, X_test, y_test)
print(f'MAE: {mae}, MSE: {mse}, RMSE: {rmse}, R²: {r2}')


209/209 [==============================] - 1s 3ms/step
Mean Absolute Error: 0.05919130923496448
Mean Squared Error: 0.013959264114422965
Root Mean Squared Error: 0.11814932972481462
R² Score: 0.961965621953317
MAE: 0.05919130923496448, MSE: 0.013959264114422965, RMSE: 0.11814932972481462, R²: 0.961965621953317


In [3]:
mae, mse, rmse, r2 = Model.evaluate_model(lstm_model_wo_water, X_test, y_test)
print(f'MAE: {mae}, MSE: {mse}, RMSE: {rmse}, R²: {r2}')

209/209 [==============================] - 1s 3ms/step
Mean Absolute Error: 0.060965433321353624
Mean Squared Error: 0.013864753132669164
Root Mean Squared Error: 0.11774868633097
R² Score: 0.9622231331215363
MAE: 0.060965433321353624, MSE: 0.013864753132669164, RMSE: 0.11774868633097, R²: 0.9622231331215363


In [4]:

# Create dataset for Scikit-learn model
X_train_lr, y_train_lr = Model.create_sklearn_dataset(normalized_data, n_future, n_past)
X_test_lr, y_test_lr = Model.create_sklearn_dataset(normalized_data, n_future, n_past)

# Build, train, and evaluate the linear regression model
lr_model, mae, mse, rmse, r2 = Model.sklearn_autoregressive_linear_model(X_train_lr, y_train_lr, X_test_lr, y_test_lr)

# Print the evaluation metrics
print(f'Linear Regression - MAE: {mae}, MSE: {mse}, RMSE: {rmse}, R²: {r2}')

Scikit-learn Autoregressive Linear Model - MAE: 0.24981182799756652, MSE: 0.14879831988206543, RMSE: 0.38574385268214634, R²: 0.5943023153739073
Linear Regression - MAE: 0.24981182799756652, MSE: 0.14879831988206543, RMSE: 0.38574385268214634, R²: 0.5943023153739073


In [6]:
# Prepare the dataset
X_train_ar, y_train_ar = Model.create_sklearn_dataset(normalized_data, n_past, n_future)
X_test_ar, y_test_ar = Model.create_sklearn_dataset(normalized_data, n_past, n_future)

# Train and evaluate the Scikit-learn autoregressive model
sklearn_ar_model, sklearn_ar_mae, sklearn_ar_mse, sklearn_ar_rmse, sklearn_ar_r2 = Model.sklearn_autoregressive_linear_model(X_train_ar, y_train_ar, X_test_ar, y_test_ar)



Scikit-learn Autoregressive Linear Model - MAE: 0.322071460074731, MSE: 0.21284586873724234, RMSE: 0.46135221765722806, R²: 0.4196770756728394
